In [38]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

# 7 Assests (Nvidia, Advanced Micro Devices, Gamestop, Walt Disney Co, Pfizer inc, Exxon Mobil Corp, Target Corp)
tickers =['NVDA','AMD','GME','DIS','PFE','XOM','TGT']

# Gathers asset's data from past 10 years (365 days) to today 
start_date = dt.datetime.today() - pd.Timedelta(3650, "d")
end_date = dt.datetime.today()

#Downloads stock data
tickers_data =yf.download (start=start_date,end=end_date, tickers= ['NVDA','AMD','GME','DIS','PFE','XOM','TGT'])['Adj Close']
tickers_data.head()


[*********************100%%**********************]  7 of 7 completed


Ticker,AMD,DIS,GME,NVDA,PFE,TGT,XOM
Date,,,,,,,
2014-03-28,3.88,72.459625,7.378824,4.240737,20.765766,45.145355,63.407005
2014-03-31,4.01,73.450340,7.466017,4.243108,20.922089,45.544262,63.394035
2014-04-01,4.07,74.826332,7.734868,4.418424,20.811357,45.589424,63.426476
2014-04-02,4.06,74.918053,7.878376,4.389995,21.032822,45.830273,63.569248
2014-04-03,4.00,74.936401,8.063663,4.437376,21.104471,46.454998,63.556267


In [39]:
daily_returns = tickers_data.pct_change()
daily_returns= daily_returns.drop(index = daily_returns.index[0], axis=0, inplace=False) # drops row with NaN values: the value of the first row because the numerator = 0 when calculating daily returns since there is no data from previous day to calculate for the next day.
print(daily_returns)

Ticker           AMD       DIS       GME      NVDA       PFE       TGT  \
Date                                                                     
2014-03-31  0.033505  0.013673  0.011817  0.000559  0.007528  0.008836   
2014-04-01  0.014963  0.018734  0.036010  0.041318 -0.005293  0.000992   
2014-04-02 -0.002457  0.001226  0.018554 -0.006434  0.010642  0.005283   
2014-04-03 -0.014778  0.000245  0.023518  0.010793  0.003407  0.013631   
2014-04-04  0.002500 -0.015424 -0.034918 -0.030966 -0.007407 -0.009397   
...              ...       ...       ...       ...       ...       ...   
2024-03-19 -0.048413  0.005797 -0.035945  0.010661 -0.003247  0.003700   
2024-03-20 -0.009315  0.017029  0.005220  0.010895  0.002534  0.013376   
2024-03-21 -0.005842  0.002061  0.013353  0.011762 -0.001444 -0.001701   
2024-03-22  0.005429 -0.007112 -0.040996  0.031213 -0.010846 -0.009050   
2024-03-25 -0.005678  0.030120  0.154198  0.007562  0.002558  0.023898   

Ticker           XOM  
Date          

LECTURE NOTES:
For returns, use Adjusted Close instead of Close to normalize everything to it's current price point.
%change is a function for Rolling
To calculate return : Assestname*(today's price)-Assetnames*(yesterday's price)/Assetnames*(yesterday's price)
This gives the mean daily returns based on Adj Close price. 
The NaN values do not compute a % change for row 2019-02-26 because there are no values for 2019-02-25 to roll onto the next day.
The following rows are the % change from the 26 to the 27, then the 27 to 28 and so on. Rolling period defualt is one day
Using portfolio.pct_change().mean() calculates the mean for each column. 


In [40]:
# Column 3: Anualized volatility: trailing 3 months
#Source: https://www.youtube.com/watch?v=j4c2XqiJzRU

#Assuming average days for 3-month period = 90 days
days = 90
# Calculates annualized volatility trailing over 3-months 
volatility_three = daily_returns.rolling(window=days).std() * np.sqrt(days)*100

# Drop NaN values from the trailing window
volatility_three = volatility_three.dropna()
volatility_three

Ticker,AMD,DIS,GME,NVDA,PFE,TGT,XOM
Date,,,,,,,
2014-08-06,28.080182,9.136591,21.106673,13.554195,9.694822,11.391197,7.869216
2014-08-07,27.894109,9.145901,21.133990,13.592303,9.683249,11.380917,7.904106
2014-08-08,27.860128,9.090044,20.826783,15.685433,9.747753,11.529675,8.041141
2014-08-11,27.861675,9.114918,20.872970,15.680716,9.677364,11.519602,8.046630
2014-08-12,27.860090,9.124103,20.802610,15.645510,9.677527,11.435025,8.051055
...,...,...,...,...,...,...,...
2024-03-19,29.901105,17.839358,44.105365,26.787281,16.356256,24.828670,10.878474
2024-03-20,29.935664,17.883467,44.021126,26.783010,16.307844,24.770606,10.795244
2024-03-21,29.950622,16.618551,43.802099,26.785968,16.137308,24.552579,10.799234


In [41]:
#Beta against an asset= the covariance(daily returns of assest)/variance(daily returns of what is being compared to: SPY,IWM or DIA)
#Beta = how a stock varies compared to current market
    #A measure of risk: 
    #   B= 1, volatile as market.
    #   B>1, more volatile. 
    #   B<1, less volatile. 

source for Columns 4-6: https://www.youtube.com/watch?v=_8DZxoEwpwE

In [42]:
#Column4: Beta against SPY
assets = ['SPY','NVDA','AMD','GME','DIS','PFE','XOM','TGT']

start_date2 = dt.datetime.today() - pd.Timedelta(365, "d")
end_date2 = dt.datetime.today()

ticker2 = yf.download(assets,start=start_date2, end=end_date2)
ticker2=ticker2['Adj Close']
ticker2.head()


[*********************100%%**********************]  8 of 8 completed


Ticker,AMD,DIS,GME,NVDA,PFE,SPY,TGT,XOM
Date,,,,,,,,
2023-03-27,96.610001,95.309380,23.000000,265.224945,38.251911,390.830322,154.211121,102.144966
2023-03-28,94.559998,94.511978,23.150000,264.015320,38.033169,389.953033,154.588470,103.419365
2023-03-29,96.089996,96.555321,22.459999,269.753479,38.280441,395.620972,152.982315,105.195793
2023-03-30,97.879997,97.781319,22.500000,273.742188,38.404079,397.937439,155.072266,105.707489
2023-03-31,98.010002,99.804726,23.020000,277.680908,38.803532,403.546204,160.258423,105.871620


In [43]:
data= pd.DataFrame(ticker2)
data.head()

Ticker,AMD,DIS,GME,NVDA,PFE,SPY,TGT,XOM
Date,,,,,,,,
2023-03-27,96.610001,95.309380,23.000000,265.224945,38.251911,390.830322,154.211121,102.144966
2023-03-28,94.559998,94.511978,23.150000,264.015320,38.033169,389.953033,154.588470,103.419365
2023-03-29,96.089996,96.555321,22.459999,269.753479,38.280441,395.620972,152.982315,105.195793
2023-03-30,97.879997,97.781319,22.500000,273.742188,38.404079,397.937439,155.072266,105.707489
2023-03-31,98.010002,99.804726,23.020000,277.680908,38.803532,403.546204,160.258423,105.871620


In [44]:
#Calculate daily returns 
daily_returns2 = ticker2.pct_change()
print(daily_returns2)


Ticker           AMD       DIS       GME      NVDA       PFE       SPY  \
Date                                                                     
2023-03-27       NaN       NaN       NaN       NaN       NaN       NaN   
2023-03-28 -0.021219 -0.008366  0.006522 -0.004561 -0.005718 -0.002245   
2023-03-29  0.016180  0.021620 -0.029806  0.021734  0.006501  0.014535   
2023-03-30  0.018628  0.012697  0.001781  0.014786  0.003230  0.005855   
2023-03-31  0.001328  0.020693  0.023111  0.014388  0.010401  0.014095   
...              ...       ...       ...       ...       ...       ...   
2024-03-19 -0.048413  0.005797 -0.035945  0.010661 -0.003247  0.005557   
2024-03-20 -0.009315  0.017029  0.005220  0.010895  0.002534  0.009249   
2024-03-21 -0.005842  0.002061  0.013353  0.011762 -0.001444  0.003305   
2024-03-22  0.005429 -0.007112 -0.040996  0.031213 -0.010846 -0.001896   
2024-03-25 -0.005678  0.030120  0.154198  0.007562  0.002558 -0.002763   

Ticker           TGT       XOM  
Date

In [45]:
#Covariances and Variance
covariance= daily_returns2.cov()
variance= np.var(daily_returns2['SPY'])
beta =[] 

for i in assets:
    asset_beta = covariance.loc[i,'SPY']/variance
    beta.append(asset_beta)

beta

[1.0040160642570275,
 2.246214263754308,
 2.1181439456555684,
 1.624606583764686,
 0.8915203026293439,
 0.3590090133078715,
 0.3914485157275384,
 0.7109968564300155]

In [46]:
df=pd.DataFrame({'Asset': assets, 'Beta against SPY':beta})
df=df.round(2)
df

,Asset,Beta against SPY
0,SPY,1.00
1,NVDA,2.25
2,AMD,2.12
3,GME,1.62
4,DIS,0.89
5,PFE,0.36
6,XOM,0.39
7,TGT,0.71


In [47]:
#Column 5: Beta against IWM

assets2 = ['IWM','NVDA','AMD','GME','DIS','PFE','XOM','TGT']

#start_date2 = dt.datetime.today() - pd.Timedelta(365, "d")
#end_date2 = dt.datetime.today()

ticker3 = yf.download(assets2,start=start_date2, end=end_date2)
ticker3=ticker3['Adj Close'] #daily price
ticker3.head()


[*********************100%%**********************]  8 of 8 completed


Ticker,AMD,DIS,GME,IWM,NVDA,PFE,TGT,XOM
Date,,,,,,,,
2023-03-27,96.610001,95.309380,23.000000,171.346207,265.224945,38.251911,154.211121,102.144966
2023-03-28,94.559998,94.511978,23.150000,171.099594,264.015320,38.033165,154.588455,103.419373
2023-03-29,96.089996,96.555321,22.459999,173.042892,269.753479,38.280441,152.982315,105.195793
2023-03-30,97.879997,97.781319,22.500000,172.806137,273.742188,38.404079,155.072266,105.707481
2023-03-31,98.010002,99.804726,23.020000,175.982513,277.680908,38.803528,160.258423,105.871620


In [48]:
data2= pd.DataFrame(ticker3)
data2.head()

daily_returns3 = ticker3.pct_change()
print(daily_returns3)


Ticker           AMD       DIS       GME       IWM      NVDA       PFE  \
Date                                                                     
2023-03-27       NaN       NaN       NaN       NaN       NaN       NaN   
2023-03-28 -0.021219 -0.008366  0.006522 -0.001439 -0.004561 -0.005719   
2023-03-29  0.016180  0.021620 -0.029806  0.011358  0.021734  0.006502   
2023-03-30  0.018628  0.012697  0.001781 -0.001368  0.014786  0.003230   
2023-03-31  0.001328  0.020693  0.023111  0.018381  0.014388  0.010401   
...              ...       ...       ...       ...       ...       ...   
2024-03-19 -0.048413  0.005797 -0.035945  0.004224  0.010661 -0.003247   
2024-03-20 -0.009315  0.017029  0.005220  0.019792  0.010895  0.002534   
2024-03-21 -0.005842  0.002061  0.013353  0.011490  0.011762 -0.001444   
2024-03-22  0.005429 -0.007112 -0.040996 -0.013802  0.031213 -0.010846   
2024-03-25 -0.005678  0.030120  0.154198  0.002292  0.007562  0.002558   

Ticker           TGT       XOM  
Date

In [49]:
covariance2= daily_returns3.cov()
variance2= np.var(daily_returns3['IWM'])
beta2 =[] 

for i in assets2:
    asset_beta2 = covariance2.loc[i,'IWM']/variance2
    beta2.append(asset_beta2)
print(beta2)

[1.0040160642570286, 0.5032609834295678, 0.6528942844468749, 1.480931931642592, 0.5443064407637251, 0.21981236695569123, 0.37507472900809846, 0.5719601610902729]


In [50]:
df2=pd.DataFrame({'Asset': assets2, 'Beta against IWM':beta2})
df2=df2.round(2)
print(df2)

  Asset  Beta against IWM
0   IWM              1.00
1  NVDA              0.50
2   AMD              0.65
3   GME              1.48
4   DIS              0.54
5   PFE              0.22
6   XOM              0.38
7   TGT              0.57


In [51]:
#Column 6: Beta against DIA

assets3 = ['DIA','NVDA','AMD','GME','DIS','PFE','XOM','TGT']

ticker4 = yf.download(assets3,start=start_date2, end=end_date2)
ticker4=ticker4['Adj Close'] #daily price
ticker4.head()

data3= pd.DataFrame(ticker4)
data3.head()


[*********************100%%**********************]  8 of 8 completed


Ticker,AMD,DIA,DIS,GME,NVDA,PFE,TGT,XOM
Date,,,,,,,,
2023-03-27,96.610001,318.143341,95.309380,23.000000,265.224945,38.251911,154.211121,102.144966
2023-03-28,94.559998,317.652863,94.511978,23.150000,264.015320,38.033165,154.588470,103.419365
2023-03-29,96.089996,320.860657,96.555321,22.459999,269.753479,38.280441,152.982315,105.195793
2023-03-30,97.879997,322.234100,97.781319,22.500000,273.742188,38.404083,155.072250,105.707481
2023-03-31,98.010002,326.295319,99.804726,23.020000,277.680908,38.803528,160.258423,105.871628


In [52]:
daily_returns4 = ticker4.pct_change()
print(daily_returns4)

covariance3= daily_returns4.cov()
variance3= np.var(daily_returns4['DIA'])
beta3 =[] 

for i in assets3:
    asset_beta3 = covariance3.loc[i,'DIA']/variance3
    beta3.append(asset_beta3)
print(beta3)

Ticker           AMD       DIA       DIS       GME      NVDA       PFE  \
Date                                                                     
2023-03-27       NaN       NaN       NaN       NaN       NaN       NaN   
2023-03-28 -0.021219 -0.001542 -0.008366  0.006522 -0.004561 -0.005719   
2023-03-29  0.016180  0.010098  0.021620 -0.029806  0.021734  0.006502   
2023-03-30  0.018628  0.004280  0.012697  0.001781  0.014786  0.003230   
2023-03-31  0.001328  0.012603  0.020693  0.023111  0.014388  0.010401   
...              ...       ...       ...       ...       ...       ...   
2024-03-19 -0.048413  0.008176  0.005797 -0.035945  0.010661 -0.003247   
2024-03-20 -0.009315  0.010463  0.017029  0.005220  0.010895  0.002534   
2024-03-21 -0.005842  0.006988  0.002061  0.013353  0.011762 -0.001444   
2024-03-22  0.005429 -0.008096 -0.007112 -0.040996  0.031213 -0.010846   
2024-03-25 -0.005678 -0.004005  0.030120  0.154198  0.007562  0.002558   

Ticker           TGT       XOM  
Date

In [53]:
df3=pd.DataFrame({'Asset': assets3, 'Beta against DIA':beta3})
df3=df3.round(2)
df3

,Asset,Beta against DIA
0,DIA,1.00
1,NVDA,1.24
2,AMD,1.25
3,GME,1.80
4,DIS,1.16
5,PFE,0.59
6,XOM,0.83
7,TGT,1.07


In [54]:
#Column 7: Average Weekly drawdown 
#   ([52_weeks_Low - 52_weeks_High])/(52_weeks_High)
tickers =['NVDA','AMD','GME','DIS','PFE','XOM','TGT']

start_date3 = dt.datetime.today() - pd.Timedelta(365, "d")
end_date3 = dt.datetime.today()

portfolio2 =yf.download(start=start_date3,end =end_date3, tickers= ['NVDA','AMD','GME','DIS','PFE','XOM','TGT'])['Adj Close']
portfolio2.head()

[*********************100%%**********************]  7 of 7 completed


Ticker,AMD,DIS,GME,NVDA,PFE,TGT,XOM
Date,,,,,,,
2023-03-27,96.610001,95.309380,23.000000,265.224945,38.251911,154.211136,102.144966
2023-03-28,94.559998,94.511978,23.150000,264.015320,38.033165,154.588486,103.419365
2023-03-29,96.089996,96.555321,22.459999,269.753479,38.280445,152.982315,105.195793
2023-03-30,97.879997,97.781319,22.500000,273.742188,38.404079,155.072266,105.707481
2023-03-31,98.010002,99.804726,23.020000,277.680908,38.803528,160.258438,105.871620


In [55]:
returns = portfolio2.pct_change() #daily returns
returns= returns.dropna()

#Changes frequency from daily to weekly returns
weekly_returns = returns.resample('W-Fri').last()

print("Weekly Returns: ")
weekly_returns.head()

Weekly Returns: 


Ticker,AMD,DIS,GME,NVDA,PFE,TGT,XOM
Date,,,,,,,
2023-03-31,0.001328,0.020693,0.023111,0.014388,0.010401,0.033444,0.001553
2023-04-07,-0.000972,0.000601,0.014952,0.005803,-0.001203,0.002058,-0.016583
2023-04-14,-0.003692,-0.009322,-0.003991,0.011148,-0.006752,-0.013903,0.002419
2023-04-21,-0.018644,0.015295,0.013353,0.000554,0.009034,0.011854,0.003200
2023-04-28,0.022072,0.025718,0.018479,0.019209,0.003872,-0.002529,0.012925


In [56]:
#2 Calculate Cumulative Returns
cumulative_max= weekly_returns.cummax()
#3 weekly drawdowns for 1y period=52 weeks
weekly_drawdown= (weekly_returns-cumulative_max)/ cumulative_max
weekly_drawdown.head()

Ticker,AMD,DIS,GME,NVDA,PFE,TGT,XOM
Date,,,,,,,
2023-03-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-07,-1.732042,-0.970978,-0.353021,-0.596679,-1.115678,-0.938474,-11.679364
2023-04-14,-3.779688,-1.450467,-1.172693,-0.225241,-1.649138,-1.415728,0.000000
2023-04-21,-15.036882,-0.260859,-0.422220,-0.961526,-0.131457,-0.645545,0.000000
2023-04-28,0.000000,0.000000,-0.200410,0.000000,-0.627753,-1.075626,0.000000


In [57]:
# This loop gives the avg for each weekly drawndown column
avg_drawdown=[]

#for i in weekly_drawdown:
 #   avg_drawdown= weekly_drawdown['AMD'].mean(),weekly_drawdown['DIS'].mean(),weekly_drawdown['GME'].mean(),weekly_drawdown['NVDA'].mean(),weekly_drawdown['PFE'].mean(),weekly_drawdown['TGT'].mean(),weekly_drawdown['XOM'].mean()

for i in tickers:
    avg_drawdown= weekly_drawdown[tickers].mean()  
avg_drawdown=avg_drawdown.round(4)

avg_drawdown


Ticker
NVDA   -0.8756
AMD    -1.1894
GME    -0.9813
DIS    -0.9404
PFE    -0.9074
XOM    -1.0742
TGT    -1.0330
dtype: float64

In [58]:
#Column 8: Maximum Weekly Drawdown 
#Source: https://quant.stackexchange.com/questions/18094/how-can-i-calculate-the-maximum-drawdown-mdd-in-python
#        https://medium.com/cloudcraftz/measuring-maximum-drawdown-and-its-python-implementation-99a3963e158f

weekly_returns.head()
window=365
Roll_Max = weekly_returns.rolling(window, min_periods=1).max()
max_weekly_drawdown = weekly_returns/Roll_Max - 1.0

max_drawdown_values = max_weekly_drawdown.min()
max_drawdown_values=max_drawdown_values.round(4)

max_drawdown_values


Ticker
AMD    -15.0369
DIS     -1.9479
GME     -2.0801
NVDA    -2.1160
PFE     -2.5212
TGT     -2.2030
XOM    -11.6794
dtype: float64

In [59]:
#Column 9: Total Return trailing 10 years

start_date = dt.datetime.today() - pd.Timedelta(3650, "d")
end_date = dt.datetime.today()

#Downloads stock data
portfolio3 =yf.download (start=start_date,end=end_date, tickers= ['NVDA','AMD','GME','DIS','PFE','XOM','TGT'])['Adj Close']
portfolio3.inplace=True

portfolio3.tail()


[*********************100%%**********************]  7 of 7 completed


Ticker,AMD,DIS,GME,NVDA,PFE,TGT,XOM
Date,,,,,,,
2024-03-19,181.419998,114.510002,13.41,893.979980,27.629999,168.210007,113.089996
2024-03-20,179.729996,116.459999,13.48,903.719971,27.700001,170.460007,112.989998
2024-03-21,178.679993,116.699997,13.66,914.349976,27.660000,170.169998,113.489998
2024-03-22,179.649994,115.870003,13.10,942.890015,27.360001,168.630005,113.489998
2024-03-25,178.630005,119.360001,15.12,950.020020,27.430000,172.660004,114.650002


In [60]:
current_price= (portfolio3.iloc[-1]) # Returns the end date data for each asset 
current_price

Ticker
AMD     178.630005
DIS     119.360001
GME      15.120000
NVDA    950.020020
PFE      27.430000
TGT     172.660004
XOM     114.650002
Name: 2024-03-25 00:00:00, dtype: float64

In [61]:
portfolio3.head()


Ticker,AMD,DIS,GME,NVDA,PFE,TGT,XOM
Date,,,,,,,
2014-03-28,3.88,72.459618,7.378820,4.240739,20.765760,45.145348,63.406986
2014-03-31,4.01,73.450340,7.466016,4.243107,20.922094,45.544273,63.394028
2014-04-01,4.07,74.826317,7.734866,4.418425,20.811359,45.589424,63.426453
2014-04-02,4.06,74.918060,7.878373,4.389995,21.032827,45.830273,63.569233
2014-04-03,4.00,74.936424,8.063662,4.437376,21.104475,46.455002,63.556278


In [62]:
starting_price= portfolio3.iloc[0] #returns the starting price for each asset
starting_price

Ticker
AMD      3.880000
DIS     72.459618
GME      7.378820
NVDA     4.240739
PFE     20.765760
TGT     45.145348
XOM     63.406986
Name: 2014-03-28 00:00:00, dtype: float64

In [63]:
for i in tickers:
    total_return=(((current_price-starting_price)/starting_price)*100).round(2)
    
total_return

Ticker
AMD      4503.87
DIS        64.73
GME       104.91
NVDA    22302.23
PFE        32.09
TGT       282.45
XOM        80.82
dtype: float64

In [64]:
#Column 10 : Annualized total return trailing 10 years
annual_return = (((1+total_return)**(365/3650)-1)*100).round(2)
annual_return

Ticker
AMD     131.94
DIS      51.98
GME      59.40
NVDA    172.17
PFE      41.90
TGT      75.89
XOM      55.34
dtype: float64

In [65]:
#Table 1

analysis = pd.DataFrame(index=tickers)

#Portfolio Wieght (must be equally weighted!) = 1/7 (since there are seven assests)
analysis['Portfolio Weight']= 1/len(tickers)

#Annualized Volatility (trailing 3 months), iloc[-1] slices the last row/ current data from each column.
analysis['Annualized Volatility'] = volatility_three.iloc[-1]

#Betas againts SPY,IWM and DIA
analysis = analysis.merge(df, left_index=True, right_on='Asset', how='left').set_index('Asset')
analysis = analysis.merge(df2, left_index=True, right_on='Asset', how='left').set_index('Asset')
analysis = analysis.merge(df3, left_index=True, right_on='Asset', how='left').set_index('Asset')

#Avg Weekly Drawdown
analysis['Avg Weekly Drawdown']= avg_drawdown

#Max Weekly Drawdown
analysis['Max Weekly Drawdown']= max_drawdown_values

#Total Returns
analysis['Total Returns']= total_return

#Annualized Total Return
analysis['Annualized Total Return']= annual_return
print('Table 1: ')
analysis

Table 1: 


,Portfolio Weight,Annualized Volatility,Beta against SPY,Beta against IWM,Beta against DIA,Avg Weekly Drawdown,Max Weekly Drawdown,Total Returns,Annualized Total Return
Asset,,,,,,,,,
NVDA,0.142857,26.799844,2.25,0.50,1.24,-0.8756,-2.1160,22302.23,172.17
AMD,0.142857,29.633980,2.12,0.65,1.25,-1.1894,-15.0369,4503.87,131.94
GME,0.142857,46.427634,1.62,1.48,1.80,-0.9813,-2.0801,104.91,59.40
DIS,0.142857,16.632888,0.89,0.54,1.16,-0.9404,-1.9479,64.73,51.98
PFE,0.142857,16.075483,0.36,0.22,0.59,-0.9074,-2.5212,32.09,41.90
XOM,0.142857,10.778621,0.39,0.38,0.83,-1.0742,-11.6794,80.82,55.34
TGT,0.142857,24.602032,0.71,0.57,1.07,-1.0330,-2.2030,282.45,75.89


In [66]:
# Table 2
print ("testing") 


testing
